# General Observation

In [4]:
import pandas as pd
data_klaim = pd.read_csv("/kaggle/input/datasets/fathinahnurjannah/mcf-dsc-itb-cleaned/data_klaim_cleaned.csv")
print(data_klaim.head())

   claim_id nomor_polis reimburse/cashless inpatient/outpatient icd_diagnosis  \
0  C-0002-M    POL-3288                  R                   OP           C34   
1  C-0003-M    POL-1786                  R                   OP         C18.9   
2  C-0004-M    POL-1786                  R                   OP           C34   
3  C-0011-M    POL-2516                  R                   OP         L02.2   
4  C-0012-M    POL-2794                  C                   IP           A09   

                                     icd_description status_klaim  \
0            MALIGNANT NEOPLASM OF BRONCHUS AND LUNG         PAID   
1             MALIGNANT NEOPLASM, COLON, UNSPECIFIED         PAID   
2            MALIGNANT NEOPLASM OF BRONCHUS AND LUNG         PAID   
3  CUTANEOUS ABSCESS, FURUNCLE AND CARBUNCLE OF T...         PAID   
4  OTHER GASTROENTERITIS AND COLITIS OF INFECTIOU...         PAID   

  tanggal_pembayaran_klaim tanggal_pasien_masuk_rs tanggal_pasien_keluar_rs  \
0               202

In [5]:
data_polis = pd.read_csv("/kaggle/input/datasets/fathinahnurjannah/mcf-dsc-itb-cleaned/data_polis_cleaned.csv")
print(data_polis.head())

  nomor_polis plan_code gender tanggal_lahir tanggal_efektif_polis domisili
0    POL-0001     M-003      M    1964-08-11            2014-06-03  JAKARTA
1    POL-0002     M-003      M    1971-07-30            2014-06-03  JAKARTA
2    POL-0003     M-001      M    1979-08-21            2016-08-08  JAKARTA
3    POL-0004     M-003      M    2014-07-24            2016-08-11  JAKARTA
4    POL-0005     M-001      F    1981-01-14            2015-08-28  JAKARTA


In [28]:
print(data_polis.dtypes)

nomor_polis              string[python]
plan_code                      category
gender                         category
tanggal_lahir            datetime64[ns]
tanggal_efektif_polis    datetime64[ns]
domisili                       category
dtype: object


In [29]:
print(data_klaim.dtypes)

claim_id                                 object
nomor_polis                      string[python]
reimburse/cashless                     category
inpatient/outpatient                   category
icd_diagnosis                    string[python]
icd_description                  string[python]
status_klaim                           category
tanggal_pembayaran_klaim         datetime64[ns]
tanggal_pasien_masuk_rs          datetime64[ns]
tanggal_pasien_keluar_rs         datetime64[ns]
nominal_klaim_yang_disetujui            float64
nominal_biaya_rs_yang_terjadi           float64
lokasi_rs                        string[python]
loss_ratio                              float64
los                                       Int64
icd_group                        string[python]
yearmonth                        datetime64[ns]
dtype: object


In [25]:
data_polis = data_polis.astype({
    "nomor_polis": "string",
    "plan_code" : "category",
    "gender": "category", 
    "domisili": "category"
})

data_polis["tanggal_lahir"] = pd.to_datetime(data_polis["tanggal_lahir"], format="%Y-%m-%d")
data_polis["tanggal_efektif_polis"] = pd.to_datetime(data_polis["tanggal_efektif_polis"], format="%Y-%m-%d")

In [27]:
data_klaim = data_klaim.astype({
    "nomor_polis": "string",
    "reimburse/cashless" : "category", 
    "inpatient/outpatient" : "category", 
    "icd_diagnosis": "string",
    "icd_description": "string",
    "status_klaim" : "category",
    "nominal_klaim_yang_disetujui": "float64",
    "nominal_biaya_rs_yang_terjadi": "float64",
    "lokasi_rs": "string",
    "loss_ratio": "float64",
    "los": "Int64",
    "icd_group": "string",    
})

data_klaim["tanggal_pembayaran_klaim"] = pd.to_datetime(data_klaim["tanggal_pembayaran_klaim"], format="%Y-%m-%d")
data_klaim["tanggal_pasien_masuk_rs"] = pd.to_datetime(data_klaim["tanggal_pasien_masuk_rs"], format="%Y-%m-%d")
data_klaim["tanggal_pasien_keluar_rs"] = pd.to_datetime(data_klaim["tanggal_pasien_keluar_rs"], format="%Y-%m-%d")
data_klaim["yearmonth"] = pd.to_datetime(data_klaim["yearmonth"], format="%Y-%m")
# data_klaim["yearmonth"] = data_klaim["yearmonth"].dt.to_period("M")

In [36]:
data_klaim.to_parquet("/kaggle/working/data_klaim_cleaned.parquet", index=False)

In [37]:
data_polis.to_parquet("/kaggle/working/data_polis_cleaned.parquet", index=False)

# Data Aggregation

In [39]:
import pandas as pd

# load data
data_klaim = pd.read_csv("/kaggle/input/datasets/fathinahnurjannah/mcf-dsc-itb-cleaned/data_klaim_cleaned.csv", parse_dates=["tanggal_pembayaran_klaim"])

# --- agregasi dasar ---
klaim_agg = (
    data_klaim
    .groupby("nomor_polis")
    .agg(
        total_klaim = ("nominal_klaim_yang_disetujui", "sum"),
        rata_rata_klaim = ("nominal_klaim_yang_disetujui", "mean"),
        frekuensi_klaim = ("nominal_klaim_yang_disetujui", "count"),
        terakhir_klaim = ("tanggal_pembayaran_klaim", "max"),
        perdana_klaim = ("tanggal_pembayaran_klaim", "min")
    )
    .reset_index()
)

cutoff_date = pd.Timestamp("2025-07-31")

klaim_agg["hari_sejak_klaim_terakhir"] = (
    cutoff_date - klaim_agg["terakhir_klaim"]
).dt.days


print(klaim_agg.head())

  nomor_polis   total_klaim  rata_rata_klaim  frekuensi_klaim terakhir_klaim  \
0    POL-0003  1.413816e+07     1.413816e+07                1     2024-10-31   
1    POL-0006  1.786125e+08     8.930625e+07                2     2024-12-13   
2    POL-0010  6.497528e+07     3.248764e+07                2     2024-02-27   
3    POL-0011  3.597431e+08     1.798715e+08                2     2025-08-27   
4    POL-0013  1.569392e+08     1.569392e+08                1     2025-07-24   

  perdana_klaim  hari_sejak_klaim_terakhir  
0    2024-10-31                        273  
1    2024-11-13                        230  
2    2024-02-27                        520  
3    2024-12-13                        -27  
4    2025-07-24                          7  


In [40]:
data_polis = pd.read_csv("/kaggle/input/datasets/fathinahnurjannah/mcf-dsc-itb-cleaned/data_polis_cleaned.csv")

df_model = data_polis.merge(
    klaim_agg,
    on="nomor_polis",
    how="left"
)

# isi NaN karena polis tanpa klaim
df_model[["total_klaim","rata_rata_klaim","frekuensi_klaim"]] = \
    df_model[["total_klaim","rata_rata_klaim","frekuensi_klaim"]].fillna(0) 

In [44]:
klaim_agg.to_parquet("/kaggle/working/klaim_agg.parquet", index=False)
klaim_agg.to_csv("/kaggle/working/klaim_agg.csv", index=False)

# Exposure 

In [43]:
data_polis["bulan_mulai"] = data_polis["tanggal_efektif_polis"].dt.to_period("m").dt.to_timestamp()

exposure_list = []

for m in calendar["month"]:
    exposed = data_polis[data_polis["bulan_mulai"] <= m]
    
    exposure_list.append({
        "month": m,
        "exposure": exposed["nomor_polis"].nunique()
    })

exposure_df = pd.DataFrame(exposure_list)

AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
claims_monthly = (
    data_klaim
    .assign(month=lambda x: x["claim_date"].dt.to_period("M").dt.to_timestamp())
    .groupby("month")
    .agg(
        frekuensi_klaim=("frekuensi_klaim","count"),
        total_klaim=("total_klaim","sum")
    )
    .reset_index()
) 